# Table01
Table of available DWD stations with corresponding altitude and station tags.

Annual summed hours of clear sky detected by the Bright-Sun algorithm are shown in comparison to summed sunshine duration hours according to WMO.

Further, the number of days feasible for clear sky fitting are shown for season and whole year.

In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import configparser


from trosat import sunpos as sp
import modules.load_data as ld

config = configparser.ConfigParser()
config.read("ConfigFile.ini")
pf = config['PATHS']['datasets']
# datasets
# clear sky detection - produced by make_dataset_CSD.py
pf_csd = os.path.join(pf,'CSD/BS_{station}.nc')
# DWD irradiance observation data. - acquired from DWD (ask witthuhn@tropos.de)
pf_dwd = os.path.join(pf,'DWD/2015_{station}.nc')

# DWD station metadata
st= ld.dwd_stations()
station_names = st.station.data
tags = xr.DataArray(['$\\sim$','$\\wedge$','n','s','Cfb','Dfb','Dfc'],dims=['tag'],
                    coords=[['shore','mountain','north','south','cfb','dfb','dfc']])

For the clear sky fitting, a minimum of 2 hours clear sky is required. An hour of a day is counted as clear sky if clear sky is detected at at least 45 minutes of this hour. The Aggregation of the minute resolution clear sky mask is done by the following function: 

In [2]:
def aggregate(csd):
    """
    ### aggregate minute resolution csd mask to one hour
    ### one hour is marked as clear sky if at least 1 minute
    ### is clear sky
    """
    csdsum = csd.groupby('time.hour').sum()>1
    csdh=csdsum.astype(int)
    return csdh

header=str(" & & & & \\multicolumn{2}{c|}{CSD [h]}& & \\multicolumn{5}{c|}{CSF days} \\\\ \n"+# & \\multicolumn{5}{c}{SD [\\%]} \\\\ \n"+
           "abbr. & tags & station & altitude [m] & cloud-free & free-sun & SD [h] & DJF & MAM & JJA & SON & year \\\\ \n"+#& DJF & MAM & JJA & SON & year \\\\ \n"+
           "\\middlehline")
print("######################################################################")
print(header)
for station in station_names:
    if station =='ZG':
        continue

    # load data
    CSD = xr.load_dataset(pf_csd.format(station=station))
    DWD = xr.load_dataset(pf_dwd.format(station=station))
    
    # calculate 
    # clear sky detection from dataset
    csdc = CSD.csdc
    csds = CSD.csds
    csdch1 = csdc.groupby('time.dayofyear').map(aggregate)
    csdch1 = np.sum(csdch1,axis=1)
    CSDc_hours = np.sum(csdc).values/60.
    CSDs_hours = np.sum(csds).values/60.
    
    Rdwd=DWD.diffuse_sc/(DWD['global']-DWD.diffuse_sc) 
    idx = CSD.csds.values*(0<=Rdwd)*(Rdwd<=1)
    days,counts = np.unique(DWD.where(idx,drop=True).time.astype('datetime64[D]'),
                                return_counts=True)
        

    # CSF requirement:
    #  - at least 10 min of clear sky at the day
    #  - clear sky in two different hours of the day
    CSFdays = (csdch1>2) 
    CSFdays*= (idx.groupby('time.dayofyear').sum()>10)
    CSFdays = CSFdays.assign_coords({'time':('dayofyear',pd.date_range("2015-01-01","2015-12-31"))}).swap_dims({'dayofyear':'time'})
    CSFdays_year = CSFdays.sum()
    CSFdays_seas = CSFdays.groupby("time.season").sum()
    
    # sunshine duration (WMO: sunshine if DNI>=120Wm-2)
    DNI = (DWD['global']-DWD.diffuse_sc)/np.cos(np.deg2rad(DWD.solar_zenith))
    DNI[DNI<0]=np.nan
    SD = np.count_nonzero(DNI>=120)/60.

    
    # add tags
    line= ''
    sep = False
    for i,tag in enumerate(list(st.tag.values)):
        check = bool(st.sel(station=station,tag=tag).site.values)
        if check:
            if sep:
                line+=', '
            line+=str(tags.sel(tag=tag).values)
            sep=True
            
    # add data
    height=int(st.sel(station=station).height.values)
    name = st.sel(station=station).name.values
    
    # print latex table
    print(str(f"{station} & {line:17s} & {name:23s} & {height:4d} & "+
              f"{CSDc_hours:.1f} & {CSDs_hours:.1f} & {SD:.1f} & "+
              f"{CSFdays_seas.sel(season='DJF').values:d} & " + 
              f"{CSFdays_seas.sel(season='MAM').values:d} & " + 
              f"{CSFdays_seas.sel(season='JJA').values:d} & " + 
              f"{CSFdays_seas.sel(season='SON').values:d} & " + 
              f"{CSFdays_year.values:d} " +               
              "\\\\"))

######################################################################
 & & & & \multicolumn{2}{c|}{CSD [h]}& & \multicolumn{5}{c|}{CSF days} \\ 
abbr. & tags & station & altitude [m] & cloud-free & free-sun & SD [h] & DJF & MAM & JJA & SON & year \\ 
\middlehline
AK & $\sim$, n, Cfb    & Arkona                  &   42 & 164.0 & 417.7 & 2044.5 & 5 & 23 & 31 & 5 & 64 \\
BG & n, Cfb            & Braunschweig            &   88 & 75.2 & 205.4 & 1734.5 & 5 & 15 & 24 & 13 & 57 \\
BN & n, Cfb            & Bremen (FWW)            &    5 & 67.1 & 188.0 & 1659.4 & 3 & 16 & 17 & 8 & 44 \\
CH & Dfb               & Chemnitz                &  357 & 57.7 & 234.0 & 1936.8 & 7 & 6 & 21 & 13 & 47 \\
DN & Dfb               & Dresden-Klotzsche       &  222 & 86.0 & 260.2 & 1966.7 & 14 & 23 & 28 & 17 & 82 \\
FB & $\wedge$, Dfc     & Fichtelberg             & 1213 & 32.5 & 148.9 & 1765.6 & 1 & 4 & 7 & 6 & 18 \\
FL & $\wedge$, s, Dfb  & Fürstenzell             &  476 & 123.6 & 397.6 & 1969.9 & 11 & 15 & 38 &

# Table A3:
Relative Amount of number of days that have to be interpolated for Figure 7 and 8 for each individual station.

In [ ]:
def aggregate(csd):
    """
    ### aggregate minute resolution csd mask to one hour
    ### one hour is marked as clear sky if at least 1 minute
    ### is clear sky
    """
    csdsum = csd.groupby('time.hour').sum()>1
    csdh=csdsum.astype(int)
    return csdh

header=str(" & \\multicolumn{5}{c|}{interpolated days} \\\\ \n"+# & \\multicolumn{5}{c}{SD [\\%]} \\\\ \n"+
           "abbr. & DJF & MAM & JJA & SON & year \\\\ \n"+#& DJF & MAM & JJA & SON & year \\\\ \n"+
           "\\middlehline")
print("######################################################################")
print(header)
for station in station_names:

    if station =='ZG':
        continue

    # load data
    CSD = xr.load_dataset(pf_csd.format(station=station))
    DWD = xr.load_dataset(pf_dwd.format(station=station))
    
    # calculate 
    # clear sky detection from dataset
    csdc = CSD.csdc
    csds = CSD.csds
    csdch1 = csdc.groupby('time.dayofyear').map(aggregate)
    csdch1 = np.sum(csdch1,axis=1)
    CSDc_hours = np.sum(csdc).values/60.
    CSDs_hours = np.sum(csds).values/60.
    
    Rdwd=DWD.diffuse_sc/(DWD['global']-DWD.diffuse_sc) 
    idx = CSD.csds.values*(0<=Rdwd)*(Rdwd<=1)
    days,counts = np.unique(DWD.where(idx,drop=True).time.astype('datetime64[D]'),
                                return_counts=True)
        

    # CSF requirement:
    #  - at least 10 min of clear sky at the day
    #  - clear sky in two different hours of the day
    CSFdays = (csdch1>2) 
    CSFdays*= (idx.groupby('time.dayofyear').sum()>10)
    CSFdays = CSFdays.assign_coords({'time':('dayofyear',pd.date_range("2015-01-01","2015-12-31"))}).swap_dims({'dayofyear':'time'})
    CSFdays_year = CSFdays.sum()
    CSFdays_seas = CSFdays.groupby("time.season").sum()
    
    # mark all days including interpolated days as True,
    # for calculation of relative fraction of interpolated days for 
    # figures 7 and 8
    idstart = np.argmax(CSFdays.values)
    idend = - np.argmax(CSFdays.values[::-1])
    if idend==0:
        idend=None
    CSFdays.values[idstart:idend]=True
    CSFdays_seasfull=(CSFdays).groupby('time.season').sum()
    


    
    # add tags
    line= ''
    sep = False
    for i,tag in enumerate(list(st.tag.values)):
        check = bool(st.sel(station=station,tag=tag).site.values)
        if check:
            if sep:
                line+=', '
            line+=str(tags.sel(tag=tag).values)
            sep=True
            
    # add data
    height=int(st.sel(station=station).height.values)
    name = st.sel(station=station).name.values
    
    # print latex table
    print(str(f"{station} &"+
              
              f"{100.*(1.-CSFdays_seas/CSFdays_seasfull).sel(season='DJF').values:.0f} & " + 
              f"{100.*(1.-CSFdays_seas/CSFdays_seasfull).sel(season='MAM').values:.0f} & " + 
              f"{100.*(1.-CSFdays_seas/CSFdays_seasfull).sel(season='JJA').values:.0f} & " + 
              f"{100.*(1.-CSFdays_seas/CSFdays_seasfull).sel(season='SON').values:.0f} & " + 
              f"{100.*(1.-CSFdays_year.values/365.):.0f} " +               
              "\\\\"))